In [24]:
import pandas as pd
import numpy as np
from scipy import spatial

In [3]:
from predictNew import PreprocessData

In [ ]:
dist_df

In [4]:
data = PreprocessData()

In [114]:
total_df = data.transform('data/GLASSTEST.csv', 'data/ETHANTEST.csv')

In [93]:
class Predict():

    def __init__(self, df):
        self.df = df

    def prompt_user(self, df, index):
        recommended_posting = pd.DataFrame(df.iloc[index, :]).T
        original_index = int(recommended_posting['indices'])
        label_index = self.df[self.df['indices'] == original_index].index[0]
        print(recommended_posting.iloc[0,3])
        label = input("\nyes/no (quit)")
        if label == 'quit':
            return "Done"
        if label == 'yes':
            self.df.iat[label_index,4] = 1.0
        if label == 'no':
            self.df.iat[label_index,4] = -1.0
        return self.df

    def improve_yes(self, df):
        yes_df = pd.DataFrame(df[df['labels'] == 1.0])
        yes_remains = pd.DataFrame(df[df['labels'] == 0.0])
        yes_remains = yes_remains.set_index(np.arange(0, yes_remains.shape[0]))
        distances_from_yes = np.zeros((yes_remains.shape[0], yes_df.shape[0]))
        for i in range(yes_df.shape[0]):
            for j in range(yes_remains.shape[0]):
                distances_from_yes[j,i] = (spatial.distance.cosine(yes_df.iloc[i,6:], yes_remains.iloc[j,6:]))
        yes_remains['distances_from_yes'] = np.sum(distances_from_yes, axis=1)
        return yes_remains

    def improve_no(self, df):
        no_df = pd.DataFrame(df[df['labels'] == -1.0])
        no_remains = pd.DataFrame(df[df['labels'] == 0.0])
        no_remains = no_remains.set_index(np.arange(0, no_remains.shape[0]))
        distances_from_no = np.zeros((no_remains.shape[0], no_df.shape[0]))
        for i in range(no_df.shape[0]):
            for j in range(no_remains.shape[0]):
                distances_from_no[j,i] = (spatial.distance.cosine(no_df.iloc[i,6:], no_remains.iloc[j,6:]))
        no_remains['distances_from_no'] = np.sum(distances_from_no, axis=1)
        return no_remains

    def find_next_best(self, df, index=1):
        adjusted_df = self.prompt_user(df, index)
        try:
            yes_remains = self.improve_yes(adjusted_df)
            no_remains = self.improve_no(adjusted_df)
        except TypeError:
            return self.convert_to_csv()
        total_distance = yes_remains.distances + yes_remains.distances_from_yes - no_remains.distances_from_no
        remains = yes_remains.copy()
        remains['total_distance'] = total_distance
        next_best_index = remains.total_distance.idxmin()
        return self.find_next_best(remains, next_best_index)

    def convert_to_csv(self):
        check_variable = input('Proceed? (yes / no)')
        if check_variable == 'yes':
            name = input('Enter name: ')
            self.df.to_csv('/data/%s.csv' % name)
        else:
            return self.df

    def transform(self, df):
        try:
            self.find_next_best(df)
        except ValueError:
            return self.convert_to_csv()


In [405]:
T = np.zeros(10)

In [406]:
T.fill(np.nan)

In [407]:
T

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [392]:
i = 0
j = 0
next_best_index = 0

In [431]:
sample_df = pd.DataFrame(total_df.iloc[1:6, :])
sample_df = sample_df.set_index(np.arange(0, sample_df.shape[0]))

In [432]:
sample_df.iat[0, 4] = 1.0

In [433]:
sample_df

,indices,titles,companies,jobs,labels,distances,0,1,2,3,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,2,Data Scientist,"3.6 ★Tableau Software– Seattle, Washington",**What youll be doing**The primary focus of th...,1.0,0.861950,0.0,0.000000,0.000000,0.066963,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.036914,0.066963,0.000000,0.0
1,8,Senior Data Scientist,"5.0 ★Suplari– Seattle, WA",Our mission at Suplari is to shine the light o...,0.0,0.871546,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.026564,0.000000,0.000000,0.0
2,1,Data Scientist,"3.8 ★Amazon– Seattle, WA",Job Description\nThe Amazon Demand Forecasting...,0.0,0.899140,0.0,0.042778,0.042778,0.000000,...,0.0,0.042778,0.042778,0.042778,0.0,0.000000,0.047164,0.000000,0.000000,0.0
3,5,Data Scientist I,"3.8 ★Expedia– Bellevue, Washington",Expedia\n\nWho we are\n\neCP Expedia eCommerce...,0.0,0.902016,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.043292,0.000000,0.000000,0.043292,0.0
4,3,"Data Scientist, Sr.","3.7 ★Seattle Children's Hospital– Seattle, WA",Join our Bioinformatics Delivery Team! Using t...,0.0,0.911966,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.084110,0.000000,0.000000,0.0


In [452]:
yes_dist = pd.DataFrame(np.zeros(sample_df.shape[0]))
yes_dist.replace(0, np.nan, inplace=True)
no_dist = pd.DataFrame({})

In [453]:
yes_dist

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [454]:
remains_df = pd.DataFrame(sample_df[sample_df['labels'] == 0.0])

In [455]:
list(remains_df.index)

[1, 2, 3, 4]

In [456]:
yes_df = pd.DataFrame(sample_df[sample_df['labels'] == 1.0])
yes_df = yes_df.set_index(np.arange(0, yes_df.shape[0]))

In [457]:
yes_df

,indices,titles,companies,jobs,labels,distances,0,1,2,3,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,2,Data Scientist,"3.6 ★Tableau Software– Seattle, Washington",**What youll be doing**The primary focus of th...,1.0,0.86195,0.0,0.0,0.0,0.066963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.036914,0.066963,0.0,0.0


In [460]:
for k in remains_df.index:
    print(k)

1
2
3
4


In [461]:
i

0

In [459]:
for k in remains_df.index:
    yes_dist.iloc[k,i] = (spatial.distance.cosine(yes_df.iloc[i,6:], remains_df.iloc[k,6:]))

IndexError: single positional indexer is out-of-bounds

In [388]:
recommended_posting = pd.DataFrame(sample_df.iloc[next_best_index, :]).T
original_index = int(recommended_posting['indices'])
label_index = sample_df[sample_df['indices'] == original_index].index[0]
print(recommended_posting.iloc[0,3])
label = input("\nyes/no (quit)")
if label == 'yes':
    sample_df.iat[label_index,4] = 1.0
    remains_df = pd.DataFrame(sample_df[sample_df['labels'] == 0.0])
    remains_df = remains_df.set_index(np.arange(0, remains_df.shape[0]))
    yes_dist[i] = np.zeros(sample_df.shape[0])
    yes_df = pd.DataFrame(sample_df[sample_df['labels'] == 1.0])
    yes_df = yes_df.set_index(np.arange(0, yes_df.shape[0]))
    for k in range(remains_df.index):
        yes_dist.iloc[k,i] = (spatial.distance.cosine(yes_df.iloc[i,6:], remains_df.iloc[k,6:]))
    i += 1
if label == 'no':
    sample_df.iat[label_index,4] = -1.0
    remains_df = pd.DataFrame(sample_df[sample_df['labels'] == 0.0])
    remains_df = remains_df.set_index(np.arange(0, remains_df.shape[0]))
    no_dist[i] = np.zeros(remains_df.shape[0])
    no_df = pd.DataFrame(sample_df[sample_df['labels'] == -1.0])
    no_df = no_df.set_index(np.arange(0, no_df.shape[0]))
    for k in range(remains_df.shape[0]):
        no_dist.iloc[k,j] = (spatial.distance.cosine(no_df.iloc[j,6:], remains_df.iloc[k,6:]))
    j += 1

Job Description
The Amazon Demand Forecasting team seeks a Data Scientist with strong analytical and communication skills to join our team. We develop sophisticated algorithms that involve learning from large amounts of data, such as prices, promotions, similar products, and a product’s attributes, in order to forecast the demand of over 190 million products world-wide. These forecasts are used to automatically order more than $200 million worth of inventory weekly, establish labor plans for tens of thousands of employees, and predict the company’s financial performance. The work is complex and important to Amazon. With better forecasts we drive down supply chain costs, enabling the offer of lower prices and better in-stock selection for our customers.

In a typical day, you will work closely with talented machine learning scientists, statisticians, software engineers, and business groups. Your work will include cutting edge technologies that enable implementation of sophisticated mode


yes/no (quit) yes


ValueError: Length of values does not match length of index

In [389]:
i

1

In [390]:
j

1

In [370]:
sample_df

,indices,titles,companies,jobs,labels,distances,0,1,2,3,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,2,Data Scientist,"3.6 ★Tableau Software– Seattle, Washington",**What youll be doing**The primary focus of th...,1.0,0.861950,0.0,0.000000,0.000000,0.066963,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.036914,0.066963,0.000000,0.0
1,8,Senior Data Scientist,"5.0 ★Suplari– Seattle, WA",Our mission at Suplari is to shine the light o...,-1.0,0.871546,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.026564,0.000000,0.000000,0.0
2,1,Data Scientist,"3.8 ★Amazon– Seattle, WA",Job Description\nThe Amazon Demand Forecasting...,0.0,0.899140,0.0,0.042778,0.042778,0.000000,...,0.0,0.042778,0.042778,0.042778,0.0,0.000000,0.047164,0.000000,0.000000,0.0
3,5,Data Scientist I,"3.8 ★Expedia– Bellevue, Washington",Expedia\n\nWho we are\n\neCP Expedia eCommerce...,0.0,0.902016,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.043292,0.000000,0.000000,0.043292,0.0
4,3,"Data Scientist, Sr.","3.7 ★Seattle Children's Hospital– Seattle, WA",Join our Bioinformatics Delivery Team! Using t...,0.0,0.911966,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.084110,0.000000,0.000000,0.0


In [371]:
remains_df

,indices,titles,companies,jobs,labels,distances,0,1,2,3,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,1,Data Scientist,"3.8 ★Amazon– Seattle, WA",Job Description\nThe Amazon Demand Forecasting...,0.0,0.899140,0.0,0.042778,0.042778,0.0,...,0.0,0.042778,0.042778,0.042778,0.0,0.000000,0.047164,0.0,0.000000,0.0
1,5,Data Scientist I,"3.8 ★Expedia– Bellevue, Washington",Expedia\n\nWho we are\n\neCP Expedia eCommerce...,0.0,0.902016,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.043292,0.000000,0.0,0.043292,0.0
2,3,"Data Scientist, Sr.","3.7 ★Seattle Children's Hospital– Seattle, WA",Join our Bioinformatics Delivery Team! Using t...,0.0,0.911966,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.084110,0.0,0.000000,0.0


In [372]:
yes_df

,indices,titles,companies,jobs,labels,distances,0,1,2,3,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,2,Data Scientist,"3.6 ★Tableau Software– Seattle, Washington",**What youll be doing**The primary focus of th...,1.0,0.86195,0.0,0.0,0.0,0.066963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.036914,0.066963,0.0,0.0


In [373]:
no_df

,indices,titles,companies,jobs,labels,distances,0,1,2,3,...,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309
0,8,Senior Data Scientist,"5.0 ★Suplari– Seattle, WA",Our mission at Suplari is to shine the light o...,-1.0,0.871546,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.026564,0.0,0.0,0.0


In [374]:
yes_df.shape[0]

1

In [375]:
no_df.shape[0]

1

In [376]:
remains_df.shape[0]

3

In [391]:
yes_dist

,0
0,0.744379
1,0.779748
2,0.774839
3,0.774536


In [378]:
no_dist

,1
0,0.803968
1,0.766617
2,0.790635


In [379]:
dist_df = pd.DataFrame({})

In [380]:
dist_df['indices'] = remains_df['indices']
dist_df['yes_sum'] = np.sum(yes_dist, axis=1)
dist_df['no_sum'] = np.sum(no_dist, axis=1)

In [382]:
dist_df = dist_df.fillna(0)

In [383]:
dist_df['total_dist'] = remains_df['distances'] + dist_df['yes_sum'] - dist_df['no_sum']

In [384]:
dist_df

,indices,yes_sum,no_sum,total_dist
0,1,0.744379,0.803968,0.839552
1,5,0.779748,0.766617,0.915147
2,3,0.774839,0.790635,0.896169


In [385]:
index_min = dist_df['indices'][dist_df['total_dist'].idxmin()]

In [386]:
next_best_index = sample_df[sample_df['indices'] == index_min].index[0]

In [387]:
next_best_index

2